# 提示词工程介绍
提示词工程是指**设计**和**优化**对大模型的输入内容，其目的让模型能得到一个最优的输出，这是一个不断试错、需要通过大量练习获取经验和值得努力的一个过程。以下将用阿里的百炼大模型平台，介绍提示词工程的一些概念

## Tokenization
我们跟大模型交互时使用的是文本数据，但大模型却不是直接于原始的文本数据打交道。在文本进入大模型前会有一个叫做token序列化的过程，它将文本转换成对应的token（也是数字），进行分析和结果输入，同样的大模型在训练时也是使用token作为语料训练的，所以查看输入的文本如何转换成token的将直接影响到大模型的输出结果。
对于OPENAI模型，可以使用[OpenAI Tokenizer ](https://platform.openai.com/tokenizer?WT.mc_id=academic-105485-koreyst)，对于千问这样的模型，可以使用Hugging Face transformers包里的AutoTokenizer实现（对应的模型名称可在[Hugging Facing官网](https://huggingface.co/Qwen)上查到）具体如下：

In [ ]:
from transformers import AutoTokenizer

# This script is used to load a tokenizer from the Hugging Face Transformers library.

# 加载预训练模型对应的分词器
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-14B")

# 进行分词处理（英文）
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

tokens = tokenizer(text)

# 输出分词对应的token id
print("Tokens:", tokens['input_ids'])

# 输出分词结果
[print(f"b[: {tokenizer.decode(token_id)}]") for token_id in tokens['input_ids']]

可以使用不同的模型，查看一下不同的分词结果。千问也是一个中文模型，那来看一下中文分词结果是如何的：

In [ ]:
from transformers import AutoTokenizer

# This script is used to load a tokenizer from the Hugging Face Transformers library.

# 加载预训练模型对应的分词器
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-14B")

# 进行分词处理（英文）
text = f"""
木星是距离太阳的第五颗行星，也是太阳系中最大的行星。\
它是一颗气态巨行星，质量是太阳的千分之一，但却是太阳系所有其他行星总和的两倍半。\
木星是夜空中肉眼可见的最亮的天体之一，自有记载历史之前就为古代文明所熟知。它以罗马神朱庇特的名字命名。[19]\
从地球上看，木星的亮度足以使其反射的光线投射出可见的阴影，[20]并且平均是夜空中仅次于月球和金星的第三亮自然物体。
"""

tokens = tokenizer(text)

# 输出分词对应的token id
print("Tokens:", tokens['input_ids'])

# 输出分词结果
[print(f"b[: {tokenizer.decode(token_id)}]") for token_id in tokens['input_ids']]

跟英文不同，它是根据词来划分的，而英文可以把一个单词拆分到两个token中

## 连接API KEY
如果在本地要与厂商提供的大模型进行交互，需要使用厂商提供的API key。一般在练习的时候，需要将API key放入系统环境中，，操作[在此](https://bailian.console.aliyun.com/?tab=api#/api/?type=model&url=2803795)描述。以下是测试API key有没有加载到系统环境中的测试代码，如果成功，会有一段补全文字数据：

In [ ]:
import os
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"), # 环境变量
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

## Updated
def get_completion(prompt, model="qwen-plus"):
    messages = [{"role": "user", "content": prompt}]       
    response = client.chat.completions.create(   
        model=model,                                         
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens=1024
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt)
print(response)

## 为什么要使用提示词工程
虽然用了数据量超大的语料训练这些模型，但是由于语料时序性、大模型自身能力、不同大模型能力差异很大等限制，导致大模型会出现幻觉（提供假信息）、同一输入不同输出等问题。其实重复执行上面代码，对于同一模型（qwen-plus），输入相同得到的输出不同。以下使用不同模型（qwen-plus、deepseek），针对一个不存在的命题（2076年火星战争），设置一个课程计划，其会得到不同的结果，并且也是基于错误得到的答案。

In [ ]:
# use qwen1.5-0.5b-chat model
text = f"""
generate a lesson plan on the Martian War of 2076.
"""
prompt = f"""
```{text}```
"""
response = get_completion(prompt, model="qwen1.5-0.5b-chat")
print(response)

In [ ]:
# use deepseek model
response = get_completion(prompt, model="deepseek-v3")
print(response)

对于最近的一些模型（如qwen-plus、deepseek-v3），其对这个不存在问题的幻觉是比较少的，它们给出了一个科幻小说的课程，总体来说是可以接受的。当使用一些较早模型或性能相对较弱的模型时（如qwen1.5-0.5b-chat），就会出现幻觉，当成了一个真实的历史事件进行设计，这显然是不对的，容易让使用的人认为这是真的，不符合Responsible for AI的理念。